# Reshaping without MultiIndex

Files needed = (dogs.csv, Metro_Median_RentalPrice_1Bedroom.csv)

Our last notebook taught us about MultiIndex. The MultiIndex is a great way to organize our data and it provided an easy way to reshape our data: moving some variables from rows to columns and vice-versa. 

In this notebook, we learn about two related methods `pivot_table()` and `melt()` which allow us to reshape our data without using a MultiIndex. Sometimes, we want to reshape the data before we deal with indexing, and pivot and melt can save us some work.

In [ ]:
import pandas as pd                 # load pandas and shorten it to pd
import datetime as dt               # load datetime and shorten it to dt
import matplotlib.pyplot as plt     # for making figures

## Panel data
Panel data have (at least) three dimensions, for example, a panel might record
1. several variables (income, age, spending on cheese)
2. regarding several people
3. over time

Let's revisit the data on my dogs --- this time, without MultiIndex.

In [ ]:
# load a data file with the number of walks and snacks my dogs have had 
dogs = pd.read_csv('dogs.csv')         # data on the habits of my dogs
dogs


This data format is called **long** because there are lots of rows and not many columns. Moving between long and **wide** (lots of columns, fewer rows) is a common task in setting up panel data sets. 

Pandas calls long data **stacked** and wide data **unstacked**.

**With a MultiIndex:**
 We use the `stack()` and `unstack()` methods for moving between long and wide with MultiIndexed data. Stack and unstack do not work in place. They always return a copy, so we need to assign it to variables.

**Without a MultiIndex:**
We use the `pivot_table()` and `melt()` methods to move between long and wide data.

### From long to wide
We start by pivoting the data to wide from long (the [docs](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html#pandas.DataFrame.pivot)). It moves rows into columns.

Instead of referencing indexes, we are referencing columns in the long data. We tell pivot_table which column will be the index of the wide data, which columns will be the columns of the wide data, and which column hold the values of the columns. 

In [ ]:
# Make time variables datetime objects. Always a good practice.
dogs['time'] = pd.to_datetime(dogs['time'])

# Make th 
dogs_wide = dogs.pivot_table(index='time', columns=['dog', 'var'], values='value')
dogs_wide

A nice result. We basically jumped right to a panel. Notice that the order in which we pass columns determines the level of the MultiIndex. 

Now that I have a MultiIndexed 'wide' DataFrame, I turn it into a 'long' DataFrame frame using `stack()`, since `stack()` knows what to do with multiple indexes. 

In [ ]:
dogs_long = dogs_wide.stack(['dog', 'var'])
dogs_long

### From wide to long
The `melt()` method gets us from a wide DataFrame to a long DataFrame (the [docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.melt.html#pandas.DataFrame.melt)). It moves columns into rows.

Let's use the data from Zillow on one bedroom rental prices by metro area, 'Metro_Median_RentalPrice_1Bedroom.csv'.

In [ ]:
onebeds = pd.read_csv('Metro_MedianRentalPrice_1Bedroom.csv')
onebeds.head()

These data are wide. There is only one variable here: the median rental price of a one bedroom apartment. 

Use `melt()` to create a long DataFrame. We provide melt with the id_vars, which tell it the variables to group the data by. 

In [ ]:
onebeds_long = onebeds.melt(id_vars=['RegionName', 'SizeRank']) 
onebeds_long.head()


I'm a little surprised this didn't create a MultiIndex on the row axis, like pivot( ) did on the column axis above. We can certainly turn it into a MultiIndex... 

In [ ]:
onebeds_long.set_index(['RegionName', 'variable'], inplace=True)
onebeds_long.head()

In [ ]:
# might as well clean up the column names
onebeds_long.rename(columns={'value':'onebed_price', 'SizeRank':'size_rank'}, inplace=True)
onebeds_long.head()

In [ ]:
# and the index names
onebeds_long.index.set_names('region', level=0, inplace=True)
onebeds_long.index.set_names('date', level=1, inplace=True)
onebeds_long.head()

Now there is a nice and clean dataset. 

## Practice

Let's try some more reshaping. Take a few minutes and try the following. Feel free to chat with those around if you get stuck. The TA and I are here, too.

1. Load 'broadband.csv'. The file contains gdp per capita and broadband penetration data for several countries. Are the data long or wide?

2. Set country and series as the index.

3. Create a DataFrame with broadband and gdppc as columns, country as rows.

4. Reload the broadband data file to a new DataFrame.
5. Create the DataFrame you have for part 3., but do it using melt or pivot_table.

6. Make a scatter plot of gdp per capita against broadband penetration.

7. Save your DataFrame as 'broadband_wide.csv'

8. Read 'broadband_wide.csv' into a DataFrame named `bb_wide`

9. Turn bb_wide into a long dataset using pivot_table/melt